### Differential Evolution on Gas Transmission Pipe

### DE/current to rand/1

In [391]:
# Import library yang dibutuhkan
import numpy as np

### Differential evolution method

In [392]:
# Fungsi untuk Mutation
# Input : target pipe dan F
# Output : mutated vector
def mutation(x, F):
    return np.add(x[0], np.add(np.multiply(F, np.subtract(x[1], x[2])), np.multiply(F, np.subtract(x[3], x[4]))))

In [393]:
# Fungsi untuk Crossover
# Input : mutated vector, target pipe, dimensi dari pipe = 7, dan CR
# Output : trial vector
def crossover(mutated, target, dims, cr):
    # Generate uniform random value untuk setiap dimension
    p = np.random.rand(dims)
    # Generate Trial Vector dari binomial crossover
    trial = [mutated[i] if p[i] < cr else target[i] for i in range(dims)]
    return trial

### Methods

In [394]:
# Fungsi untuk mengecek batasan dari pd, ps, L, D pada tiap pipe dari 1 set pipe
# Input : mutated pipe dan batasan
# Output : mutated pipe yang memenuhi batasan
def check_bounds(mutated, bounds):
    mutated_bound = []
    for i in range(len(mutated)):
        temp = []
        for j in range(len(bounds)):
            temp.append(np.clip(mutated[i][j], bounds[j, 0], bounds[j, -1]))
        mutated_bound.append(temp)
    return mutated_bound

In [395]:
# Fungsi untuk cek constraint pertidaksamaan
# 1.) Pd/Ps >= 1
# 2.) Pd/Ps <= Ki
# 3.) Flow rate kuadrat = 0
# Input : target pipe
# Output : pipe yang memenuhi constraint
def check_inequality(pop, K=[]):
	# Cek Pd >= Ps
	for p in pop:
		if p[0] <= p[1]:
			p[0],p[1] = p[1],p[0]
	# Cek K
	# Cek Q
	return pop
def check_inequality_2(pop):
	ps_init = 500
	for i in range(len(pop)):
		if i == 0:
			if (pop[i][0]/ps_init) < 1:
				return False
		else:
			if (pop[i][0]/pop[i-1][1]) < 1:
				return False
	return True

In [396]:
# Fungsi untuk cek total panjang dari branch 1 dan branch 2
def check_length1(pop):
    return True if round(sum(pop[:,2])) == 175 else False
# Fungsi untuk cek total panjang dari branch 1 dan branch 3
def check_length2(pop):
    return True if round(sum(pop[:,2])) == 200 else False

### Process

Bentuk dari populasi awal untuk setiap pipe:</br>
pipe1 = [[pd, ps, l, d]1, [pd, ps, l, d]2, [pd, ps, l, d]3, ..., [pd, ps, l, d]1000]</br>
pipe2 = [[pd, ps, l, d]1, [pd, ps, l, d]2, [pd, ps, l, d]3, ..., [pd, ps, l, d]1000]</br>
pipe3 = [[pd, ps, l, d]1, [pd, ps, l, d]2, [pd, ps, l, d]3, ..., [pd, ps, l, d]1000]</br>
...</br>
pipe11 = [[pd, ps, l, d]1, [pd, ps, l, d]2, [pd, ps, l, d]3, ..., [pd, ps, l, d]1000]</br>
  
Setelah diolah DE, output yang diharapkan yaitu vector paling optimal:</br>
pipe1 = [pd, ps, l, d]</br>
pipe2 = [pd, ps, l, d]</br>
pipe3 = [pd, ps, l, d]</br>
...</br>
pipe11 = [pd, ps, l, d]</br>

In [397]:
# Definisi parameter untuk Differential Evolution
# ------------------------------
pop_size = 2000 # size population candidate solution
iter = 100  # number iteration

F = 0.02     # scale factor [0, 1]
CR = 0.8    # crossover rate [0, 2]

n = 10  # number compressor
m = n+1 # number pipe
N1 = 4  # number compressor in branch 1
N2 = 3  # number compressor in branch 2
N3 = 3  # number compressor in branch 3

In [398]:
# Definisi batas atas dan batas bawah untuk setiap dimensi pipe
# pipe = [pd, ps, l, d]

# Bounds for branch 1
bounds1 = np.asarray([
    (590, 1000),(300, 900),(2, 70),(4, 36)
])
# Bounds for branch 2
bounds2 = np.asarray([
    (590, 1000),(300, 900),(2, 30),(4, 18)
])
# Bounds for branch 3
bounds3 = np.asarray([
    (590, 1000),(200, 860),(2, 70),(4, 18)
])

In [399]:
# Fungsi Q untuk menghitung Flow Rate
# Input : pd, ps, L, D
# Output : Q
def q(pd, ps, l, d):
    return (871 * ((d)**(8/3))) * np.sqrt((pd**2 - ps**2) / l)

In [400]:
# Objective Function F()
# x = [pd, ps, l, d]

# Fungsi objektif untuk operating cost compressor
def f_comp(x, q, co = 8, cc = 70.00, T = 520, k = 1.26, z = 0.9):
  return ((co + cc) * q * 0.08531 * T * (k/(k - 1)) * ((x[0]/x[1])**((z*(k - 1))/k) - 1))
# Fungsi objektif untuk maintenance cost pipe
def f_pipe(x, cs = 870):
  return (cs * x[2] * x[3])

# Fungsi objektif
# Input: 1 set pipe (pipe1, pipe2, ..., pipei), jumlah compressor, jumlah pipe
# Output: sum fungsi objektif
def f(pop, n, m):
    sum = 0
    for i in range(n):
      qi = q(pop[i][0],pop[i][1],pop[i][2],pop[i][3])/1000000
      sum += f_comp(pop[i], qi)
      for j in range(m):
          sum += f_pipe(pop[j])
    return sum

#### Branch 1 + Branch 2

In [401]:
# Fungsi untuk mengolah populasi awal pipa dari branch 1 dan branch 2 yang hanya memenuhi panjang total
# Input : populasi awal pipe branch 1 + branch 2, ukuran populasi, N1, dan N2
# Output : vector populasi awal yang memenuhi panjang dari branch 1 dan branch 2
def get_pipe_length1(pipe_b1_b2, pop_size, N1, N2):
    best_pipe = []
    for i in range(pop_size): # 1, 2, ..., 1000
        temp = []
        sum_pipe = 0
        for j in range(N1+N2): # 1, 2, 3, 4, 5, 6, 7
            sum_pipe += pipe_b1_b2[j][i][2]
            temp.append(pipe_b1_b2[j][i])
        if(round(sum_pipe) == 175):
            best_pipe.append(temp)
    return best_pipe

In [402]:
# Proses inisialisasi populasi awal setiap pipe pada branch 1 dan branch 2
# <-----Branch 1-----> <--------Branch 2--------->
# pipe1, pipe2, pipe3, pipe4, pipe5, pipe6, pipe7,

while True:
  # Initialize population pipe branch 1
  pipe_b1 = []
  for i in range(N1-1):
    pipe_b1.append(bounds1[:, 0] + (np.random.rand(pop_size, len(bounds1)) * (bounds1[:, 1] - bounds1[:, 0])))
  # Initialize population pipe branch 2
  pipe_b2 = []
  for i in range(N2+1):
    pipe_b2.append(bounds2[:, 0] + (np.random.rand(pop_size, len(bounds2)) * (bounds2[:, 1] - bounds2[:, 0])))
  # Concatenate population pipe branch 1 and branch 2
  pipe_b1_b2 = np.concatenate((pipe_b1, pipe_b2))
  # Get all pipes whichs meet constraint length branch 1 and branch 2
  best_pipe_b1_b2 = get_pipe_length1(pipe_b1_b2, pop_size, N1, N2)
  if(best_pipe_b1_b2 != []):
    break

In [403]:
# Cek berapa populasi awal vector yang memenuhi panjang total
len(best_pipe_b1_b2)

31

In [404]:
# Fungsi differential evolution untuk pipe branch 1 dan branch 2
def diff_evol1(pop_pipe, iter, pop_size, bounds, F, CR):

    # Check inequality
    pop_pipe = [check_inequality(p) for p in pop_pipe]
    pop_pipe = np.array(pop_pipe)

    # Evaluate initial population candidate solution
    obj_all = [f(p, N1+N2, N1+N2) for p in pop_pipe]

    # Find the best vector of initial population
    best_vector = pop_pipe[np.argmin(obj_all)]
    best_obj = min(obj_all)
    prev_obj = best_obj

    for i in range(iter):
        # Iterate over all candidate solution
        for j in range(pop_size):
            # MUTATION
            while(True):
                # Choose 3 candidate solution : a, b, c, d, e.
                a = pop_pipe[j] # current
                c = pop_pipe[j] # current
                # Choose 5 candidate solution : a, b, c, d, e
                candidates = [candidate for candidate in range(pop_size) if candidate != j]
                d, e, b = pop_pipe[np.random.choice(candidates, 3, replace=False)]
                # Perform mutation
                mutated = mutation([a, b, c, d, e], F)
                # Check bound mutated vector
                mutated_b1 = check_bounds(mutated[:3], bounds[0])
                mutated_b2 = check_bounds(mutated[3:], bounds[1])
                mutated = np.concatenate((mutated_b1, mutated_b2))
                # Check inequality const
                mutated = check_inequality(mutated)
                    # Check lenght, if true continue, else LOOP UNTIL
                if check_length1(mutated):
                    break

            # CROSSOVER
            # Perform crossover
            while(True):
                trial = np.array(crossover(mutated, pop_pipe[j], N1+N2, CR))
                if check_length1(trial):
                    break

            # Compute objective function value for target vector
            obj_target = f(pop_pipe[j], N1+N2, N1+N2)
            # Compute objective function value for trial vector
            obj_trial = f(trial, N1+N2, N1+N2)

            # SELECTION
            # Perform selection
            if obj_trial < obj_target:
               # Replace target vector with trial vector
               pop_pipe[j] = trial
               # Store the new obj function value
               obj_all[j] = obj_trial

        # Find best performing vector each iteration
        best_obj = min(obj_all)
        # Store the lowest obj function value
        if best_obj < prev_obj:
            best_vector = pop_pipe[np.argmin(obj_all)]
            prev_obj = best_obj

        # Print progress iteration
        # print('Iteration %d : f[%s] = %.5f' % (i, np.around(best_vector, decimals=5), best_obj))
        # print('--------------------')

    return [best_vector, best_obj]

In [457]:
# Proses Differential Evolution hingga memenuhi setiap constraint
# Hingga memenuhi Ps demand 1 = 600 psi
while(True):
    best_b1_b2 = diff_evol1(best_pipe_b1_b2, iter, len(best_pipe_b1_b2), [bounds1, bounds2], F, CR)
    status = False
    print(best_b1_b2)
    if(round(best_b1_b2[0][N1+N2-1][1]) == 600 and check_inequality_2(best_b1_b2[0])):
        break

[array([[799.92376928, 741.92231923,  64.96453341,  10.01150672],
       [731.15028876, 660.82893605,   4.51520182,   7.33695993],
       [932.87710893, 400.11248639,  64.51681864,   8.07221223],
       [817.31485354, 768.68330638,   4.23634782,  15.58721983],
       [861.36292024, 409.1218533 ,  16.1217633 ,   5.31142394],
       [771.06742098, 460.61045256,   9.81637649,  15.02181182],
       [714.76828384, 606.88936616,  10.82314345,  13.10556881]]), 10616363.162386967]
[array([[786.66484268, 726.53970823,  61.47994659,  10.24576447],
       [729.57469982, 638.98793018,   8.69361765,   8.30602731],
       [933.92592968, 441.9812761 ,  63.1262346 ,   7.18959764],
       [810.90483474, 754.00774298,   4.16861634,  15.59840449],
       [864.56300622, 411.40233418,  15.22745163,   6.07841736],
       [765.51627861, 464.79691443,  10.94031356,  14.04621579],
       [716.33498354, 586.81788114,  11.53081997,  12.55535618]]), 10329097.699152838]
[array([[777.6521442 , 741.96852898,  60.342

In [458]:
# Vector terbaik untuk branch 1 dan branch 2
best_b1_b2

[array([[782.31476372, 730.39427231,  62.83771667,  10.58628219],
        [741.82408304, 660.07250337,   6.68885882,   7.41330378],
        [924.56221048, 417.5253014 ,  65.19846016,   8.27083763],
        [817.0781498 , 769.51322899,   4.15140332,  15.72534951],
        [865.72644378, 405.54942131,  14.85248257,   5.82336173],
        [758.29783755, 455.57982651,  10.96435362,  14.4524965 ],
        [714.79482359, 600.26798144,  10.79714611,  12.90101225]]),
 10916821.657297974]

#### Branch 1 + Branch 3

In [459]:
# Mengambil vector pipe pada branch 1 yang akan digunakan untuk evaluasi
demand1_index = 0
for i in range(len(best_b1_b2[0])):
    if round(best_b1_b2[0][i][1]) == 600:
        demand1_index = i
best_b1 = []
for p in best_b1_b2[0]:
    if(round(p[1]) != 600):
        best_b1.append(p)
    if len(best_b1) == N1-1:
        break

In [460]:
# Fungsi untuk mengolah populasi awal pipa dari branch 1 dan branch 3 yang hanya memenuhi panjang total
# Input : populasi awal pipe branch 1 + branch 3, ukuran populasi, N1, dan N3
# Output : vector populasi awal yang memenuhi panjang dari branch 1 dan branch 3
def get_pipe_length2(pipe_b3, best_b1, pop_size, N1, N3):
    best_pipe = []
    for i in range(pop_size): # 1, 2, ..., 50
        temp = []
        sum_pipe = 0
        for j in range(N1+N3): # 1, 2, 3, 4, 5, 6, 7
            if j < N1-1: # 0, 1, 2
                sum_pipe += best_b1[j][2]
            if j >= N1-1: # 3, 4, 5, 6 => 0, 1, 2, 3
                sum_pipe += pipe_b3[j-N3][i][2]
                temp.append(pipe_b3[j-N3][i])
        if(round(sum_pipe) == 200):
            best_pipe.append(temp)
    return best_pipe

In [461]:
# Proses inisialisasi populasi awal setiap pipe pada branch 3
# <-----Branch 1-----> <--------Branch 3----------->
# pipe1, pipe2, pipe3, pipe8, pipe9, pipe10, pipe11,
def create_pipe_b3():
  while True:
    # Initialize population pipe branch 3
    pipe_b3 = []
    for i in range(N3+1):
      pipe_b3.append(bounds3[:, 0] + (np.random.rand(pop_size, len(bounds3)) * (bounds3[:, 1] - bounds3[:, 0])))
    best_pipe_b3 = get_pipe_length2(pipe_b3, best_b1, pop_size, N1, N3)
    if(best_pipe_b3 != [] and len(best_pipe_b3) > 4):
      break
  return best_pipe_b3

In [462]:
def diff_evol_2(pop_pipe_b3, best_b1, iter, pop_size, bounds, F, CR):

    # Check inequality branch 3
    pop_pipe = [check_inequality(p) for p in pop_pipe_b3]
    pop_pipe = np.array(pop_pipe)

    # Evaluate initial population candidate solution
    temp = [np.concatenate((best_b1, p)) for p in pop_pipe_b3]
    obj_all = [f(p, N1+N3, N1+N3) for p in temp]

    # Find the best vector of initial population
    best_vector = pop_pipe[np.argmin(obj_all)]
    best_obj = min(obj_all)
    prev_obj = best_obj

    for i in range(iter):
        # Iterate over all candidate solution
        for j in range(pop_size):
            # MUTATION
            while(True):
                # Choose 3 candidate solution : a, b, c, d, e.
                a = pop_pipe[j] # current
                c = pop_pipe[j] # current
                # Choose 5 candidate solution : a, b, c, d, e
                candidates = [candidate for candidate in range(pop_size) if candidate != j]
                d, e, b = pop_pipe[np.random.choice(candidates, 3, replace=False)]
                # Perform mutation
                mutated = mutation([a, b, c, d, e], F)
                # Check bound mutated vector
                mutated = check_bounds(mutated, bounds)
                # Check inequality const
                mutated = check_inequality(mutated)
                # Check lenght, if true continue, else LOOP UNTIL
                temp = np.concatenate((best_b1, mutated))
                if check_length2(temp):
                    break
            # CROSSOVER
            # Perform crossover
            while(True):
                mutated_temp = np.concatenate((best_b1, mutated))
                target_temp = np.concatenate((best_b1, pop_pipe[j]))
                trial = np.array(crossover(mutated_temp, target_temp, N1+N3, CR))
                if check_length2(trial):
                    break

            # Compute objective function value for target vector]
            obj_target = f(target_temp, N1+N3, N1+N3)
            # Compute objective function value for trial vector
            obj_trial = f(trial, N1+N3, N1+N3)

            # SELECTION
            # Perform selection
            if obj_trial < obj_target:
               # Replace target vector with trial vector
               pop_pipe[j] = trial[3:]
               # Store the new obj function value
               obj_all[j] = obj_trial

        # Find best performing vector each iteration
        best_obj = min(obj_all)
        # Store the lowest obj function value
        if best_obj < prev_obj:
            best_vector = pop_pipe[np.argmin(obj_all)]
            prev_obj = best_obj

    #     # Print progress iteration
    #     # print('Iteration %d : f[%s] = %.5f' % (i, np.around(best_vector, decimals=5), best_obj))
    #     # print('--------------------')
    best_vector = np.concatenate((best_b1, best_vector))
    return [best_vector, best_obj]

In [463]:
while(True):
    best_pipe_b3 = create_pipe_b3()
    best_b1_b3 = diff_evol_2(best_pipe_b3, best_b1, iter, len(best_pipe_b3), bounds3, F, CR)
    status = False
    if(round(best_b1_b3[0][N1+N3-1][1]) == 300):
        status = True
    if status and (best_b1_b3[0][:3] == best_b1).all() and check_inequality_2(best_b1_b3[0]):
        break

In [464]:
best_b1_b3

[array([[782.31476372, 730.39427231,  62.83771667,  10.58628219],
        [741.82408304, 660.07250337,   6.68885882,   7.41330378],
        [924.56221048, 417.5253014 ,  65.19846016,   8.27083763],
        [920.07122447, 269.62352597,  39.18385288,   4.85694454],
        [744.39882086, 537.26778174,   5.7678434 ,  10.76378678],
        [766.35129776, 417.53439683,   5.79228041,  11.04804203],
        [891.7804709 , 300.48589322,  14.07690852,  12.45740474]]),
 11748859.510583859]

### Results

In [465]:
# Best vector untuk pipe di branch 1
print('Branch 1')
for i in range(len(best_b1)):
    print('Pipe {0}: {1}'.format(i+1, best_b1[i]))


Branch 1
Pipe 1: [782.31476372 730.39427231  62.83771667  10.58628219]
Pipe 2: [741.82408304 660.07250337   6.68885882   7.41330378]
Pipe 3: [924.56221048 417.5253014   65.19846016   8.27083763]


In [466]:
# Best vector untuk pipe di branch 1 dan branch 2
print('Branch 1 dan Branch 2')
for i in range(len(best_b1_b2[0])):
    print('Pipe {0}: {1}'.format(i+1, best_b1_b2[0][i]))
    if round(best_b1_b2[0][i][1]) == 600:ps1 = i
print('--------------------------------')
print('F(x) = ',best_b1_b2[1])
print('Sum L = ',sum(best_b1_b2[0][:,2]))

print('Ps == 600 terpenuhi pada pipe ke',ps1+1)


Branch 1 dan Branch 2
Pipe 1: [782.31476372 730.39427231  62.83771667  10.58628219]
Pipe 2: [741.82408304 660.07250337   6.68885882   7.41330378]
Pipe 3: [924.56221048 417.5253014   65.19846016   8.27083763]
Pipe 4: [817.0781498  769.51322899   4.15140332  15.72534951]
Pipe 5: [865.72644378 405.54942131  14.85248257   5.82336173]
Pipe 6: [758.29783755 455.57982651  10.96435362  14.4524965 ]
Pipe 7: [714.79482359 600.26798144  10.79714611  12.90101225]
--------------------------------
F(x) =  10916821.657297974
Sum L =  175.49042127256465
Ps == 600 terpenuhi pada pipe ke 7


In [467]:
# Best vector untuk pipe di branch 1 dan branch 3
print('Branch 1 dan Branch 3')
for i in range(len(best_b1_b3[0])):
    j = i+4 if i > 2 else i
    if round(best_b1_b3[0][i][1]) == 300:ps2 = j
    print('Pipe {0}: {1}'.format(j+1, best_b1_b3[0][i]))
print('--------------------------------')
print('F(x) = ',best_b1_b3[1])
print('Sum L = ',sum(best_b1_b3[0][:,2]))

print('Ps == 300 terpenuhi pada pipe ke',ps2+1)

Branch 1 dan Branch 3
Pipe 1: [782.31476372 730.39427231  62.83771667  10.58628219]
Pipe 2: [741.82408304 660.07250337   6.68885882   7.41330378]
Pipe 3: [924.56221048 417.5253014   65.19846016   8.27083763]
Pipe 8: [920.07122447 269.62352597  39.18385288   4.85694454]
Pipe 9: [744.39882086 537.26778174   5.7678434   10.76378678]
Pipe 10: [766.35129776 417.53439683   5.79228041  11.04804203]
Pipe 11: [891.7804709  300.48589322  14.07690852  12.45740474]
--------------------------------
F(x) =  11748859.510583859
Sum L =  199.54592086075704
Ps == 300 terpenuhi pada pipe ke 11


In [468]:
# Best vector untuk pipe di branch 1, branch 2, dan branch 3
best_all = np.concatenate((best_b1_b2[0],best_b1_b3[0][3:]))
print('Best vector setiap branch:')
print('\t|     Pd     |     Ps     |     L       |      D      |        Q         |')
for i in range(11):
    print('Pipe {0}: {1}'.format(i+1, best_all[i]),end="")
    if round(best_all[i][1]) == 600:ps1 = i
    if round(best_all[i][1]) == 300:ps2 = i
    print(' ',q(best_all[i][0],best_all[i][1],best_all[i][2],best_all[i][3])/1000000)
    if i == 2 or i == 6 or i == 10:
        print('---------------------------------')
print('Ps = 600 terpenuhi pada pipe ke:',ps1+1)
print('Ps = 300 terpenuhi pada pipe ke:',ps2+1)
print('---------------------------------')
print('F(x) = {0}'.format(f(best_all, n, m)))

Best vector setiap branch:
	|     Pd     |     Ps     |     L       |      D      |        Q         |
Pipe 1: [782.31476372 730.39427231  62.83771667  10.58628219]  16.638121770855854
Pipe 2: [741.82408304 660.07250337   6.68885882   7.41330378]  23.822079193544692
Pipe 3: [924.56221048 417.5253014   65.19846016   8.27083763]  24.894782940600905
---------------------------------
Pipe 4: [817.0781498  769.51322899   4.15140332  15.72534951]  182.27654015614684
Pipe 5: [865.72644378 405.54942131  14.85248257   5.82336173]  18.974239394842797
Pipe 6: [758.29783755 455.57982651  10.96435362  14.4524965 ]  197.61241401226818
Pipe 7: [714.79482359 600.26798144  10.79714611  12.90101225]  94.17898149610454
---------------------------------
Pipe 8: [920.07122447 269.62352597  39.18385288   4.85694454]  8.281142707455063
Pipe 9: [744.39882086 537.26778174   5.7678434   10.76378678]  105.54276013811717
Pipe 10: [766.35129776 417.53439683   5.79228041  11.04804203]  140.81249468224516
Pipe 11: [

In [469]:
# TODO 1. Hitung K
Ps_init = 500.00
K = []

print('Compressor Ratio')
for i in range(len(best_all)-1):
	if i == 0: K.append(best_all[i][0]/Ps_init)
	else:
		K.append(best_all[i][0]/best_all[i-1][1])
	print('Compressor {0} : {1}'.format(i+1,K[i]))

Compressor Ratio
Compressor 1 : 1.5646295274381012
Compressor 2 : 1.0156488230636325
Compressor 3 : 1.4006979623560838
Compressor 4 : 1.9569548170106443
Compressor 5 : 1.1250312680286214
Compressor 6 : 1.8698037716259939
Compressor 7 : 1.568978216314558
Compressor 8 : 1.5327674520762729
Compressor 9 : 2.760882301327338
Compressor 10 : 1.4263861035723313
